In [ ]:
import array
import numpy as np
import warnings
import scipy.sparse as sp
import itertools

from .base import BaseEstimator, ClassifierMixin, clone, is_classifier
from .base import MultiOutputMixin
from .base import MetaEstimatorMixin, is_regressor
from .preprocessing import LabelBinarizer
from .metrics.pairwise import euclidean_distances
from .utils import check_random_state
from .utils.validation import _num_samples
from .utils.validation import check_is_fitted
from .utils.validation import check_X_y, check_array
from .utils.multiclass import (_check_partial_fit_first_call,
                               check_classification_targets,
                               _ovr_decision_function)
from .utils.metaestimators import _safe_split, if_delegate_has_method

from .utils._joblib import Parallel
from .utils._joblib import delayed


In [ ]:
def _check_estimator(estimator):
    """Make sure that an estimator implements the necessary methods."""
    if (not hasattr(estimator, "decision_function") and
            not hasattr(estimator, "predict_proba")):
        raise ValueError("The base estimator should implement "
                         "decision_function or predict_proba!")

In [2]:
estimator, 
code_size=1.5
random_state=None
n_jobs=None

In [3]:
def fit(X, y):
    """Fit underlying estimators.
    Parameters
    ----------
    X : (sparse) array-like, shape = [n_samples, n_features]
        Data.
    y : numpy array of shape [n_samples]
        Multi-class targets.
    Returns
    -------
    self
    """
    X, y = check_X_y(X, y)
    if code_size <= 0:
        raise ValueError("code_size should be greater than 0, got {0}"
                         "".format(code_size))

    _check_estimator(estimator)
    random_state = check_random_state(random_state)
    check_classification_targets(y)

    classes_ = np.unique(y)
    n_classes = classes_.shape[0]
    code_size_ = int(n_classes * code_size)

    # FIXME: there are more elaborate methods than generating the codebook
    # randomly.
    code_book_ = random_state.random_sample((n_classes, code_size_))
    code_book_[code_book_ > 0.5] = 1

    if hasattr(estimator, "decision_function"):
        code_book_[code_book_ != 1] = -1
    else:
        code_book_[code_book_ != 1] = 0

    classes_index = {c: i for i, c in enumerate(classes_)}

    Y = np.array([code_book_[classes_index[y[i]]]
                  for i in range(X.shape[0])], dtype=np.int)

    estimators_ = Parallel(n_jobs=n_jobs)(
        delayed(_fit_binary)(estimator, X, Y[:, i])
        for i in range(Y.shape[1]))

    return self


In [ ]:
def predict(X):
    """Predict multi-class targets using underlying estimators.
    Parameters
    ----------
    X : (sparse) array-like, shape = [n_samples, n_features]
        Data.
    Returns
    -------
    y : numpy array of shape [n_samples]
        Predicted multi-class targets.
    """
    check_is_fitted('estimators_')
    X = check_array(X)
    Y = np.array([_predict_binary(e, X) for e in estimators_]).T
    pred = euclidean_distances(Y, code_book_).argmin(axis=1)
    return classes_[pred]